# Line following using the camera

In [ ]:
%config Completer.use_jedi = False
# To enable giving suggestions while pressing tab

import time 
import traitlets
import ipywidgets
import cv2
import numpy as np
import urllib.request
import json

from SCSCtrl import TTLServo
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot



In [ ]:
# Setting the camera direction.

# First take all servos to the initial position
def move_servos_to_initial_position():
    
    TTLServo.servoAngleCtrl(1, 0, 1, 150)
    TTLServo.servoAngleCtrl(2, 0, 1, 150)
    TTLServo.servoAngleCtrl(3, 0, 1, 150)
    TTLServo.servoAngleCtrl(4, 0, 1, 150)
    TTLServo.servoAngleCtrl(5, 0, 1, 150)

    time.sleep(2)
    # Then setting the camera direction and robot arm
    TTLServo.servoAngleCtrl(2, 50, -1, 150)
    TTLServo.servoAngleCtrl(3, 50, -1, 150)
    TTLServo.servoAngleCtrl(5, 45, 1, 150)
    TTLServo.servoAngleCtrl(1, 3, -1, 150)
    TTLServo.servoAngleCtrl(4, 25, 1, 150)

    time.sleep(3)

    TTLServo.servoStop(1)
    TTLServo.servoStop(2)
    TTLServo.servoStop(3)
    TTLServo.servoStop(5)
    TTLServo.servoStop(4)

move_servos_to_initial_position()

In [ ]:
# Displaying camera output

camera = Camera.instance(width=300, height=300)

image_widget = ipywidgets.Image()  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)


In [ ]:
def get_coordinates_from_server(server_hostname, camera_hostname, storage_id):
    try:
        url_ = f'http://{server_hostname}/AI_Service/compute_deliveryInformation?storageId={storage_id}&cameraHostname={camera_hostname}&cameraId=1'

        url_response = urllib.request.urlopen(url_)
        string = url_response.read().decode('utf-8')
        json_obj = json.loads(string)
        x = json_obj['xcoordinate']
        y = json_obj['ycoordinate']
        return x,y
    except:
        print("Error occured calling the camera AI service")
        return 0,0

def move_to_coordinate(x, y, storage_id):
    # Convert coordinates from the model
    TTLServo.servoAngleCtrl(5, 35, 1, 150)

    x_model = x
    y_model = y
#     storage_id = 3

    y_robot =  280 - y_model
    x_robot =  x_model - (storage_id - 1) * 280 - 140


    if x_robot < 0:
            num_2 = 4 - np.floor(abs(x_robot)/30)
    else:
        num_2 = 5 + np.floor(abs(x_robot)/30)
    if y_robot < 140:
        num_1 = 6 - np.floor(y_robot / 30)
    else:
        # Move robot forward
        robot.forward(0.4)
        time.sleep(2.4)
        robot.stop()
        y_robot = y_robot - 140
        num_1 = 6 - np.floor(y_robot / 30)
   
    num = num_1 * 10 + num_2
    num = num - 20
    temp = np.floor((69-num)/10) + 1
    
    h = temp * 3 - 1.5
    f = np.floor(num/10) * 10
    c = np.ceil(num/10) * 10 - 1
    if num%10 == 0:
        c = c + 10
    mid = (c + f)/2
    w = (num - mid) * 3
    h = h + 6.5


    x = np.arctan(w/h)
    x = np.degrees(x)

    l = (h ** 2 + w ** 2) ** 0.5 

    x_input = (230 - 90)/ 16 * (l - 11) + 90 
    if x < 0:
        theta = x - 6
    else:
        theta = x - 3
    TTLServo.servoAngleCtrl(1, theta, 1, 150)
    TTLServo.xyInput(x_input, -120)
    return x_input, y_model

def drop_object(x_input):
    time.sleep(0.5)
#     TTLServo.xyInput(x_input, -120)
    time.sleep(1)
    # Loose the grip
    TTLServo.servoAngleCtrl(4, 0, 1, 150)
    time.sleep(2)
    TTLServo.xyInput(x_input, -90)
    time.sleep(1)
    move_servos_to_initial_position()
    
    
        

In [ ]:


# test_mode = btntoggle.value
# test_mode = False # If test mode is enabled robot will not move 
robot_mode = "INITIALIZE"
# robot_mode = "GO" # WAIT, GO,
base_motor_speed = 0.5 # speed varies beteen 0 to 1.
# line pos 1 = 280, pos 2 = 200, pos 3 = 295
line_pos_1 = 270
line_pos_2 = 230
line_pos_3 = 230
verticle_line_pos_1 = 50
verticle_line_pos_2 = 250
line_color_set = 0
mid_point = 150
Kp = 1
Kd = 1
Ki = 0
frame_counter = 0
stop_counter = 0
storage_id = 4

EDGE_SERVER_IP = "100.66.194.7:8080"
CAMERA_IP = "100.78.195.226:8000"

robot = Robot()

previous_error = 0
I = 0

    
def pick_test_object():
    """
    This method is hard-coded to test object picking.
    """
    time.sleep(1)
    TTLServo.servoAngleCtrl(4, 20, 1, 150)
    time.sleep(2)
    TTLServo.xyInput(190, -90) # Lowering the arm
    time.sleep(2)
    TTLServo.servoAngleCtrl(4, 35, -1, 150) # Grabbing
    time.sleep(2)
    TTLServo.xyInput(100, 100) # Lifting the arm
    time.sleep(2)

def follow_line(imageInput, test_mode, dual_motor):
#     print(robot_mode)
    global previous_error, I, robot_mode, frame_counter, stop_counter, storage_id
    img = camera.value
    # plt.imshow(img)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     gray = cv2.equalizeHist(gray)
    gray_de_noised = cv2.GaussianBlur(gray, (3, 3), 0)
#     gray_de_noised = cv2.equalizeHist(gray_de_noised)
   

#     tresh = cv2.adaptiveThreshold(gray_de_noised,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
    eroded = cv2.erode(gray_de_noised, None, iterations = 8)
    dilated = cv2.dilate(eroded, None, iterations = 5)
    ret, thresh = cv2.threshold(dilated, 0, 255, cv2.THRESH_OTSU)
#     ret, thresh = cv2.threshold(dilated, 100, 255, cv2.THRESH_BINARY)

    
    
    color_pos_1 = thresh[line_pos_1]
#     print(color_pos_1)
    color_pos_2 = thresh[line_pos_2]
    color_pos_3 = thresh[line_pos_3]
    
    trimmed_line_1 = color_pos_1[verticle_line_pos_1:verticle_line_pos_2]
    trimmed_line_2 = color_pos_2[verticle_line_pos_1:verticle_line_pos_2]
    trimmed_line_3 = color_pos_3[verticle_line_pos_1:verticle_line_pos_2]
    try:
        line_color_count_pos_1 = np.sum(trimmed_line_1 == line_color_set)
#         print(line_color_count_pos_1)
        line_color_count_pos_2 = np.sum(trimmed_line_2 == line_color_set)
        
        line_index_pos_1 = np.where(trimmed_line_1 == line_color_set)
#         print(line_index_pos_1)
#         print(line_index_pos_1[0])
        line_index_pos_2 = np.where(trimmed_line_2 == line_color_set)
        
        if line_color_count_pos_1 == 0:
            line_color_count_pos_1 = 1
        if line_color_count_pos_2 == 0:
            line_color_count_pos_2 = 1
            
        right_pos_1 = line_index_pos_1[0][-1] + verticle_line_pos_1
        left_pos_1 = line_index_pos_1[0][0] + verticle_line_pos_1
        center_pos_1 = int((left_pos_1 + right_pos_1)/2)
        
        right_pos_2 = line_index_pos_2[0][-1] + verticle_line_pos_1
        left_pos_2 = line_index_pos_2[0][0] + verticle_line_pos_1
        center_pos_2 = int((left_pos_2 + right_pos_2)/2)
        
        
        center = int((center_pos_1 + center_pos_2)/2)
    except Exception as err:
#         print(err)
        center = None
        pass
    thresh = cv2.cvtColor(thresh, cv2.COLOR_GRAY2RGB)
    text = str(center)
    coordinates = (100,100)
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 1
    color = (0, 0, 255)
    thickness = 2
#     thresh = cv2.putText(thresh, text, coordinates, font, fontScale, color, thickness, cv2.LINE_AA)
#     print(center)

   
    try:
        # Box for region of interest
        cv2.line(thresh, (verticle_line_pos_1, line_pos_1), (verticle_line_pos_2, line_pos_1), (255,255,64),1)
        cv2.line(thresh, (verticle_line_pos_1, line_pos_2), (verticle_line_pos_2, line_pos_2), (255,255,64),1)

        cv2.line(thresh, (verticle_line_pos_1, line_pos_1), (verticle_line_pos_1, line_pos_2), (255,255,64),1)
        cv2.line(thresh, (verticle_line_pos_2, line_pos_1), (verticle_line_pos_2, line_pos_2), (255,255,64),1)
        
        # Middle line
        cv2.line(thresh, (150, 0), (150, 640), (0,0,255),1)
        
        # Lines for intersections of the box
        cv2.line(thresh, (left_pos_1, (line_pos_1 + 10)), (left_pos_1, (line_pos_1 - 10)), (255,128,64),1)
        cv2.line(thresh, (right_pos_1, (line_pos_1 + 10)), (right_pos_1, (line_pos_1 - 10)), (64,128,255),1)
        cv2.line(thresh, (left_pos_2, (line_pos_2 + 10)), (left_pos_2, (line_pos_2 - 10)), (255,128,64),1)
        cv2.line(thresh, (right_pos_2, (line_pos_2 + 10)), (right_pos_2, (line_pos_2 - 10)), (64,128,255),1)

        # Center marker
        cv2.line(thresh, ((center - 20), int((line_pos_1 + line_pos_2)/2)), ((center + 20), int((line_pos_1 + line_pos_2)/2)), (0,0,255), 1)
        cv2.line(thresh, ((center), int((line_pos_1 + line_pos_2)/2 + 20)), ((center), int((line_pos_1 + line_pos_2)/2 - 20)), (0,0,255), 1)

    except:
        pass
    
    if center != None:

        direction = ""
        if 145 < center < 155 :
            direction = "Go straight"
        elif center < 145 :
            direction = "Turn left"
        elif 155 < center :
            direction = "Turn right"

        trimmed_line_mid_point = int(len(trimmed_line_3)/2)
#         left_turn = np.all(trimmed_line_3[:trimmed_line_mid_point] == 0) and np.all(trimmed_line_3[-30:-10] == 255)
#         right_turn = np.all(trimmed_line_3[trimmed_line_mid_point:] == 0) and np.all(trimmed_line_3[10:30] == 255)
        left_turn = np.all(trimmed_line_3[:trimmed_line_mid_point] == 0) 
        right_turn = np.all(trimmed_line_3[trimmed_line_mid_point:] == 0)
        if robot_mode == "GO" and np.all(trimmed_line_3 == trimmed_line_3[0]):
            # Stop the robot
            robot.stop()
            direction = "Stop"
            robot_mode = "STOP"
#             print(robot_mode)
#             pick_test_object()
        elif robot_mode == "GO" and left_turn and not right_turn:
#         elif left_turn and not right_turn:
            #Turn left. Left junction
            robot_mode = "STOP"
#             print("Left")
#         elif right_turn and not left_turn:
        elif robot_mode == "GO" and right_turn and not left_turn:
            #Turn right. Right junction
#             pass
            robot_mode = "STOP"
            
        else:
            error = (center - mid_point)/mid_point
            P = error
            I = I + error
            D = error - previous_error
            PID_value = Kp*P + Kd*D + Ki*I
            previous_error = error
#             thresh = cv2.putText(thresh, str(PID_value), (150,40), font, fontScale, color, thickness, cv2.LINE_AA)


            if not test_mode:
                if robot_mode == "INITIALIZE":
                    global x,y
                    
                    robot_mode = "GO"
                elif robot_mode == "GO":
                    if dual_motor:
                        robot.left_motor.value = base_motor_speed + PID_value
                        robot.right_motor.value = base_motor_speed - PID_value
                    else:
                        if PID_value > 0:
                            robot.left_motor.value = base_motor_speed + PID_value
                            robot.right_motor.value = base_motor_speed 
                        elif PID_value <= 0 :
                            robot.left_motor.value = base_motor_speed 
                            robot.right_motor.value = base_motor_speed - PID_value
                elif robot_mode == "STOP":
                    # F = Forward, D = Drop, T = Turn/Pick, L = Left, R = Right 
                    temp_list = ['T','F','D','F','T','L','R','D','L','R','T','L','F','R','D','L','F','R','T','L','F','F','R','D' ,'S']
                    robot_mode = temp_list[stop_counter]
                    stop_counter += 1
                elif robot_mode == "T":
                    robot.stop()
                    x,y = get_coordinates_from_server(EDGE_SERVER_IP, CAMERA_IP, storage_id)
                    robot.forward(0.5)
                    time.sleep(0.4)
                    robot.stop()
                    TTLServo.servoAngleCtrl(4, 20, 1, 150)
                    time.sleep(1)
                    TTLServo.xyInput(200, -120)
                    time.sleep(3)
                    TTLServo.servoAngleCtrl(4, 45, -1, 150)
                    time.sleep(3)
                    TTLServo.xyInput(120, 70)
                    time.sleep(3)
                    
                    robot.right(0.4)
                    time.sleep(5)
                    robot.stop()
                    robot_mode = "IDLE"
                    frame_counter = 0
                     
                    # Pick object
                    # Turn
                elif robot_mode == "D":
                    robot.stop()
                    robot.forward(0.5)
                    time.sleep(0.4)
                    robot.stop()
                    x_input, y_model = move_to_coordinate(x, y, storage_id)
                    time.sleep(3)
                    # Drop object
                    drop_object(x_input)
                    storage_id -= 1
                    y_robot =  280 - y
                    if y_robot >= 140:
                        robot.backward(0.4)
                        time.sleep(2.4)
                        robot.stop()
                    robot_mode = "WAIT"
                elif robot_mode == "WAIT":
                    print("Robot waiting...")
                    robot.left(0.4)
                    time.sleep(5.1)
                    robot.stop()
                    robot_mode = "IDLE"
                    print(robot_mode)
                    frame_counter = 0
#                     time.sleep(10)
#                     robot_mode = "GO"     
                elif robot_mode == "L":
                    robot.forward(0.5)
                    time.sleep(1)
                    robot.stop()
                    robot.left(0.4)
                    time.sleep(2.7)
                    robot.stop()
                    robot_mode = "IDLE"
                    frame_counter = 0
                elif robot_mode == "R":
                    robot.forward(0.5)
                    time.sleep(1.2)
                    robot.stop()
                    robot.right(0.4)
                    time.sleep(2.7)
                    robot.stop()
                    robot_mode = "IDLE"
                    frame_counter = 0
                elif robot_mode == "F":
                    robot.forward(0.5)
                    time.sleep(2)
                    robot.stop()
                    robot_mode = "IDLE"
                    frame_counter = 0
                elif robot_mode == 'S':
                    robot.stop()
                else:
                    robot.stop()
                    # Flushing unwanted frames in the buffer.
                    if frame_counter < 10:
                        frame_counter += 1
                        robot_mode = "IDLE"
                    else:
                        robot_mode = "GO"                    
            else:
                robot.stop()  

        thresh = cv2.putText(thresh, robot_mode, (10,30), font, fontScale, color, thickness, cv2.LINE_AA)
    else:
        robot.stop()
        
    return thresh

In [ ]:
toggle1 = ipywidgets.ToggleButton(description='Test Mode')
display(toggle1)
test_mode = True
def on_click_1(change):
    global test_mode
    test_mode = change['new']
    
toggle1.observe(on_click_1, 'value')

toggle2 = ipywidgets.ToggleButton(description='Dual Motor Turn')
display(toggle2)

dual_motor = True
def on_click_2(change):
    global dual_motor
    dual_motor = change['new']
    
toggle2.observe(on_click_2, 'value')


In [ ]:
image_widget2 = ipywidgets.Image()
def execute(change):
    global image_widget2, robot_mode
#     print(robot_mode)

    image = change['new']
    image_widget.value = bgr8_to_jpeg(image)
    image_widget2.value = bgr8_to_jpeg(follow_line(image,test_mode, dual_motor))

display(image_widget2)

execute({'new': camera.value})

camera.unobserve_all()

camera.observe(execute, names='value')
   

In [ ]:
camera.unobserve(execute, names='value')

In [ ]:
camera.stop()

In [ ]:
robot.stop()

# Below code is to control the robot movements and robotic arm manually.

In [ ]:
xPos = 100
yPos = 0
servoPos_4 = 0

def xIn():
    global xPos
    xPos += 15
    TTLServo.xyInput(xPos, yPos)

def xDe():
    global xPos
    xPos -= 15
    if xPos < 85:
        xPos = 85
    TTLServo.xyInput(xPos, yPos)

def yIn():
    global yPos
    yPos += 15
    TTLServo.xyInput(xPos, yPos)

def yDe():
    global yPos
    yPos -= 15
    TTLServo.xyInput(xPos, yPos)

def grab():
    global servoPos_4
    servoPos_4 -= 15
    if servoPos_4 < -90:
        servoPos_4 = -90
    TTLServo.servoAngleCtrl(4, servoPos_4, 1, 150)
    
def loose():
    global servoPos_4
    servoPos_4+= 15
    if servoPos_4 > -10:
        servoPos_4 = -10
    TTLServo.servoAngleCtrl(4, servoPos_4, 1, 150)

arm_layout = ipywidgets.Layout(width='100px', height='80px', align_self='center')

xIn_button = ipywidgets.Button(description='X+', button_style='success', layout=arm_layout)
xDe_button = ipywidgets.Button(description='X-', button_style='success', layout=arm_layout)

yIn_button = ipywidgets.Button(description='Y+', button_style='success', layout=arm_layout)
yDe_button = ipywidgets.Button(description='Y-', button_style='success', layout=arm_layout)

grab_button = ipywidgets.Button(description='Grab', layout=arm_layout)
loose_button = ipywidgets.Button(description='Loose', layout=arm_layout)

line1_box = ipywidgets.HBox([xIn_button, yIn_button])
line2_box = ipywidgets.HBox([xDe_button, yDe_button])
line3_box = ipywidgets.HBox([grab_button, loose_button])
arm_box = ipywidgets.VBox([line1_box, line2_box, line3_box])

display(arm_box)

In [ ]:
xIn_button.on_click(lambda x: xIn())
xDe_button.on_click(lambda x: xDe())

yIn_button.on_click(lambda x: yIn())
yDe_button.on_click(lambda x: yDe())

grab_button.on_click(lambda x: grab())
loose_button.on_click(lambda x: loose())

In [ ]:

servoPos_1 = 0  # Pan position of the camera.
servoPos_5 = 0  # The vertical position of the camera.


def limitCtl(maxInput, minInput, rawInput):
    if rawInput > maxInput:
        limitBuffer = maxInput
    elif rawInput < minInput:
        limitBuffer = minInput
    else:
        limitBuffer = rawInput
        
    return limitBuffer


def cameraUp():
    global servoPos_5  # Global variables declaration
    servoPos_5 = limitCtl(25, -40, servoPos_5-15)
    TTLServo.servoAngleCtrl(5, servoPos_5, 1, 150)

def cameraDown():
    global servoPos_5  # Global variables declaration
    servoPos_5 = limitCtl(25, -40, servoPos_5+15)
    TTLServo.servoAngleCtrl(5, servoPos_5, 1, 150)

# Camera turn right
def ptRight():
    global servoPos_1
    servoPos_1 = limitCtl(50, -50, servoPos_1+15)
    TTLServo.servoAngleCtrl(1, servoPos_1, 1, 150)

# Camera turn left
def ptLeft():
    global servoPos_1
    servoPos_1 = limitCtl(50, -50, servoPos_1-15)
    TTLServo.servoAngleCtrl(1, servoPos_1, 1, 150)

In [ ]:
PT_layout = ipywidgets.Layout(width='100px', height='80px', align_self='center')
cameraUp_button = ipywidgets.Button(description='LookUp', button_style='success', layout=PT_layout)
cameraDown_button = ipywidgets.Button(description='LookDown', button_style='success', layout=PT_layout)
ptRight_button = ipywidgets.Button(description='LookRight', button_style='success', layout=PT_layout)
ptLeft_button = ipywidgets.Button(description='LookLeft', button_style='success', layout=PT_layout)

subPT_box = ipywidgets.HBox([ptLeft_button, cameraDown_button, ptRight_button], layout=ipywidgets.Layout(align_self='center'))
surPT_box = ipywidgets.VBox([cameraUp_button, subPT_box])
display(surPT_box)

In [ ]:
# Bind the button with the function.
cameraUp_button.on_click(lambda x: cameraUp())
cameraDown_button.on_click(lambda x: cameraDown())
ptRight_button.on_click(lambda x: ptRight())
ptLeft_button.on_click(lambda x: ptLeft())

In [ ]:
robot = Robot()

# create buttons
button_layout = ipywidgets.Layout(width='100px', height='80px', align_self='center')
stop_button = ipywidgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = ipywidgets.Button(description='forward', layout=button_layout)
backward_button = ipywidgets.Button(description='backward', layout=button_layout)
left_button = ipywidgets.Button(description='left', layout=button_layout)
right_button = ipywidgets.Button(description='right', layout=button_layout)

# display buttons
middle_box = ipywidgets.HBox([left_button, stop_button, right_button], layout=ipywidgets.Layout(align_self='center'))
controls_box = ipywidgets.VBox([forward_button, middle_box, backward_button])
display(controls_box)

def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.4)
#     time.sleep(0.5)
#     robot.stop()

def step_backward(change):
    robot.backward(0.4)
#     time.sleep(0.5)
#     robot.stop()

def step_left(change):
    robot.left(0.4)
#     time.sleep(0.5)
#     robot.stop()

def step_right(change):
    robot.right(0.4)
#     time.sleep(0.5)
#     robot.stop()
    
# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)